# Análise dos materiais ricos - 2017

O intuito desta análise é entender, a partir do modelo e tipo de material, qual a sua performance em relação à quantidade de download para melhor planejamento de quantidade de materiais necessárias para alcance das métricas do ano de 2018.

A conclusão ideal é que chegamos em:
- Materiais do tipo planilha geram **x** downloads e **y%** vem de forma orgânica e **z%**vem de forma paga.
- Para chegar em **x** leads, é necessário **y** materiais do tipo **z**

In [1]:
#!pip install tqdm

In [2]:
import requests
import urllib3
import numpy as np
import pandas as pd
import json
import datetime
import collections
from tqdm import tqdm

In [3]:
materiais = pd.read_csv('data/material_v2-2.csv', sep=';')

In [4]:
materiais.head()

,month+year first conv date,day first conv date,Week first conv date,month first conv date,year first conv date,first conversion date,origem,email domain analysis,Conversões,Eventos (Últimos 100)
0,01/01/2017,1,1,1,2017,01/01/2017,Desconhecido,0,3,IntegracaoExact / checklist-de-atendimento-ao-...
1,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Dados Pessoais / Criação de...
2,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Sucesso / Criação de Conta ...
3,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Sucesso / Criação de Conta ...
4,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,1,4,Criação de Conta - Sucesso / Criação de Conta ...


Afim de padronizar o nome das colunas, iremos renomeá-las para melhor operação da análise

In [5]:
materiais = materiais.rename(index=str, columns={'month+year first conv date':'month_year_conv', 'day first conv date':'day_conv',\
                                     'Week first conv date': 'week_conv', 'month first conv date': 'month_conv', \
                                     'year first conv date': 'year_conv', 'first conversion date': 'date_conv', \
                                     'origem': 'source', 'email domain analysis': 'domain', 'Conversões': 'conversions', \
                                     'Eventos (Últimos 100)': 'events'})

In [6]:
materiais.head()

,month_year_conv,day_conv,week_conv,month_conv,year_conv,date_conv,source,domain,conversions,events
0,01/01/2017,1,1,1,2017,01/01/2017,Desconhecido,0,3,IntegracaoExact / checklist-de-atendimento-ao-...
1,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Dados Pessoais / Criação de...
2,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Sucesso / Criação de Conta ...
3,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Sucesso / Criação de Conta ...
4,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,1,4,Criação de Conta - Sucesso / Criação de Conta ...


Agora, iremos quebrar a coluna 'events' para ter todos os eventos quebrados de forma unificada.

In [7]:
split_materiais = materiais['events'].str.split(' / ', expand=True)

In [8]:
materiais_final = materiais.join(split_materiais)

In [9]:
materiais_final.head()

,month_year_conv,day_conv,week_conv,month_conv,year_conv,date_conv,source,domain,conversions,events,...,90,91,92,93,94,95,96,97,98,99
0,01/01/2017,1,1,1,2017,01/01/2017,Desconhecido,0,3,IntegracaoExact / checklist-de-atendimento-ao-...,...,None,None,None,None,None,None,None,None,None,None
1,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Dados Pessoais / Criação de...,...,None,None,None,None,None,None,None,None,None,None
2,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Sucesso / Criação de Conta ...,...,None,None,None,None,None,None,None,None,None,None
3,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,0,4,Criação de Conta - Sucesso / Criação de Conta ...,...,None,None,None,None,None,None,None,None,None,None
4,01/01/2017,2,1,1,2017,02/01/2017,Desconhecido,1,4,Criação de Conta - Sucesso / Criação de Conta ...,...,None,None,None,None,None,None,None,None,None,None


In [10]:
materiais_final.drop(labels=['events'], inplace=True, axis=1)

In [11]:
materiais_final.columns

Index(['month_year_conv',        'day_conv',       'week_conv',
            'month_conv',       'year_conv',       'date_conv',
                'source',          'domain',     'conversions',
                       0,
       ...
                      90,                91,                92,
                      93,                94,                95,
                      96,                97,                98,
                      99],
      dtype='object', length=109)

In [83]:
df = materiais_final[['source', 0]]

In [84]:
df.head()

,source,0
0,Desconhecido,IntegracaoExact
1,Desconhecido,Criação de Conta - Dados Pessoais
2,Desconhecido,Criação de Conta - Sucesso
3,Desconhecido,Criação de Conta - Sucesso
4,Desconhecido,Criação de Conta - Sucesso


Criação de um dataset com todos os eventos de todos os leads, excluindo valores NaNs

In [118]:
#iteração para puxar todas as colunas para uma coluna específica
events = pd.DataFrame(data=None, dtype='object')

for x in tqdm(range(0,100)):
    df = materiais_final[['source', x]]
    df = df.rename(index=str, columns={x:'event'})
    events = pd.concat([events, df])

100%|██████████| 100/100 [00:02<00:00, 33.66it/s]


In [119]:
#dropar todos os eventos nulos
events.dropna(axis=0, inplace=True)

criar uma coluna unificando source e event para ter um controle de onde vieram os eventos e fazer a filtragem

In [120]:
events['source_event'] = events['source'] + ' | ' + events['event']

In [121]:
events['count'] = events['source_event'].map(events['source_event'].value_counts())

In [122]:
events.drop(labels=['source', 'event'], inplace=True, axis=1)

In [137]:
events['source'] = events['source_event'].str.split('|').str[0]
events['event'] = events['source_event'].str.split('|').str[1]

In [139]:
events.drop(labels=['source_event'], inplace=True, axis=1)

#### dataset final

In [141]:
events.head()

,count,event,source
0,1834,IntegracaoExact,Desconhecido
1,291,Criação de Conta - Dados Pessoais,Desconhecido
2,263,Criação de Conta - Sucesso,Desconhecido
3,263,Criação de Conta - Sucesso,Desconhecido
4,263,Criação de Conta - Sucesso,Desconhecido


In [142]:
events.to_excel('events.xlsx')